In [5]:
import sys
import os
from pathlib import Path
import itables
import pandas as pd
import datetime as dt

itables.options.warn_on_undocumented_option = False

DATA_DIR = "data"
MLRUN_ENV_FILE = "mlrun.env"

import mlrun
from config.config import load_config

pipeline_config = load_config("config/pipeline_config.yaml")

mlrun.set_env_from_file(MLRUN_ENV_FILE)

project = mlrun.get_or_create_project(
    "predictive-maintenance",
    parameters={
        "source": "s3://mlrun/predictive-maintenance.zip",
        "default_image": "felipenv/mlrun-oai:1.9",
    },
)

# project.set_secrets(
#     secrets={
#         "PG_HOST": "optimus-postgres.mlrun.svc.cluster.local",
#         "PG_USER": "optimus",
#         "PG_PASSWORD": "optimus",
#         "PG_DB": "optimus",
#     }
# )

Project Source: s3://mlrun/predictive-maintenance.zip
Exporting project as zip archive to s3://mlrun/predictive-maintenance.zip...
> 2025-09-01 19:25:56,900 [info] Project loaded successfully: {"project_name":"predictive-maintenance"}


## 0. Log Datasets + Configs

In [6]:
from datasetsforecast.phm2008 import PHM2008

In [7]:
Y_train_df, Y_test_df = PHM2008.load(directory='./data', group='FD001', clip_rul=False)


In [8]:
Y_train_df = project.log_dataset(
    key="Y_train_df", df=Y_train_df, format="csv"
)

Y_test_df = project.log_dataset(
    key="Y_test_df", df=Y_test_df, format="csv"
)

## 1. Preprocessing Function